# Code Imports

In [4]:
# Import Python Tools:
import pandas as pd
import numpy as np
from numpy import sqrt, pi, average, cos, sin, tan, arcsin, arccos, arctan, deg2rad, rad2deg
π = pi

# import sectionproperties as shape
# from WhatIfAnalysis import GoalSeek

# from ASCE import load_combos

# from steel_design.AISC import aisc, aisc_W, aisc_HSS, aisc_L
steel_density = 0.2833 #pci

# from concrete_design import material_properties as conc
# from concrete_design.steel_reinforcement import rebar_df as rebar
conc_density = 150 #pcf

# from masonry_design import material_properties as cmu
# from masonry_design.material_properties import NW_CMU #psf

<div style="page-break-after: always;"></div>

# Calculations

In [5]:
steel_density

0.2833